# CollabLLM Dataset Construction 

There are two types of datasets in CollabLLM:
- SingTurnDataset: Any single-turn tasks can be defined as a SingTurnDataset.
- MultiTurnDataset: Any multiturn conversation can be stored as MultiTurnDataset.

## Multiturn dataset

There are two ways to create a multiturn dataset:
- Provide a list of data entries (by separated rows or nested dictionary).
- Specifify a huggingface dataset repo name ([example format](https://huggingface.co/datasets/collabllm/collabllm-multiturn-math-hard)) or a local directory containing a huggingface dataset.

### Method 2.1: Create a multiturn dataset from huggingface dataset repo

In [1]:
from pprint import pprint
from datasets import DatasetDict

In [3]:
from collabllm.datasets import MultiturnDataset

ds = MultiturnDataset('collabllm/collabllm-multiturn-math-hard', add_system_prompt=True)

print("=== SFT ===")
sft_ds: DatasetDict = ds.to_sft_dataset(eval_ratio=0.1, lower_bound_metric="rewards.accuracy", lower_bound=0.5)
print(sft_ds)
print(sft_ds["train"][0])  # one example from train split

2025-08-22 11:56:09,599 [WARNING] collabllm.datasets.multiturn: Filtered out conv_id=11 (turn_id=9) due to rewards.accuracy=0.000 < 0.500
2025-08-22 11:56:09,600 [WARNING] collabllm.datasets.multiturn: Filtered out conv_id=15 (turn_id=5) due to rewards.accuracy=0.000 < 0.500
2025-08-22 11:56:09,600 [WARNING] collabllm.datasets.multiturn: Filtered out conv_id=17 (turn_id=13) due to rewards.accuracy=0.000 < 0.500
2025-08-22 11:56:09,600 [WARNING] collabllm.datasets.multiturn: Filtered out conv_id=18 (turn_id=5) due to rewards.accuracy=0.000 < 0.500
2025-08-22 11:56:09,600 [WARNING] collabllm.datasets.multiturn: Filtered out conv_id=19 (turn_id=7) due to rewards.accuracy=0.000 < 0.500
2025-08-22 11:56:09,601 [WARNING] collabllm.datasets.multiturn: Filtered out conv_id=29 (turn_id=13) due to rewards.accuracy=0.000 < 0.500
2025-08-22 11:56:09,601 [WARNING] collabllm.datasets.multiturn: Filtered out conv_id=32 (turn_id=5) due to rewards.accuracy=0.000 < 0.500
2025-08-22 11:56:09,601 [WARNING

=== SFT ===
DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 69
    })
    eval: Dataset({
        features: ['messages'],
        num_rows: 7
    })
})
{'messages': [{'content': 'The assistant is designed to be helpful, proactive, and highly interactive.\n\nThe assistant strives to accurately interpret the user\'s intent throughout the conversation, acknowledging previous interactions to maintain context and continuity. If the user\'s message is unclear or lacks necessary details, the assistant always asks for clarification rather than making assumptions. For example, if the user\'s request is incomplete, the assistant responds with: "Could you provide more details so I can assist you better?"\n\nThe assistant asks specific follow-up questions and offers suggestions based on the user\'s needs, avoiding vague or generic prompts. It proactively provides guidance and potential next steps, especially in complex tasks such as writing, analysis, coding, a

When specify DPO dataset, set `minimum_gap` to filter out pairs where the score difference is below `minimum_gap`

In [6]:
print("\n=== DPO (minimum gap = 0.1) ===")
dpo_ds: DatasetDict = ds.to_dpo_dataset(minimum_gap=0.1, eval_ratio=0.1)
print(dpo_ds)

print("\n=== DPO (minimum gap = 0.2) ===")
dpo_ds: DatasetDict = ds.to_dpo_dataset(minimum_gap=0.2, eval_ratio=0.1)
print(dpo_ds.keys())

2025-08-22 11:51:23,995 [INFO] collabllm.datasets.multiturn: Converted 250 pairs (minimum_gap=0.1, ratio=0.23)
2025-08-22 11:51:24,007 [INFO] collabllm.datasets.multiturn: Converted 188 pairs (minimum_gap=0.2, ratio=0.18)



=== DPO (minimum gap = 0.1) ===
DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected', 'score_chosen', 'score_rejected'],
        num_rows: 225
    })
    eval: Dataset({
        features: ['prompt', 'chosen', 'rejected', 'score_chosen', 'score_rejected'],
        num_rows: 25
    })
})

=== DPO (minimum gap = 0.2) ===
dict_keys(['train', 'eval'])


In [7]:
print("\n=== Inputs ===")
inputs_ds: DatasetDict = ds.to_inputs_dataset(eval_ratio=0.1)
print(inputs_ds)
print(inputs_ds["train"][0].keys())


=== Inputs ===
DatasetDict({
    train: Dataset({
        features: ['prompt', 'single_turn_prompt', 'single_turn_completion', 'single_turn_metadata'],
        num_rows: 320
    })
    eval: Dataset({
        features: ['prompt', 'single_turn_prompt', 'single_turn_completion', 'single_turn_metadata'],
        num_rows: 35
    })
})
dict_keys(['prompt', 'single_turn_prompt', 'single_turn_completion', 'single_turn_metadata'])


----

In [4]:
from collabllm.datasets import MultiturnDataset

ds = MultiturnDataset('collabllm/collabllm-multiturn-medium', add_system_prompt=True)

print("\n=== DPO (minimum gap = 0.1) ===")
dpo_ds: DatasetDict = ds.to_dpo_dataset(minimum_gap=0.1, eval_ratio=0.1)
print(dpo_ds)

print("\n=== one example from train set ===")
print(dpo_ds["train"][0])  # one example from train split

2025-08-22 14:49:10,993 [INFO] collabllm.datasets.multiturn: Converted 123 pairs (minimum_gap=0.1, ratio=0.07)



=== DPO (minimum gap = 0.1) ===
DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected', 'score_chosen', 'score_rejected'],
        num_rows: 111
    })
    eval: Dataset({
        features: ['prompt', 'chosen', 'rejected', 'score_chosen', 'score_rejected'],
        num_rows: 12
    })
})

=== one example from train set ===
{'prompt': [{'content': 'The assistant is designed to be helpful, proactive, and highly interactive.\n\nThe assistant strives to accurately interpret the user\'s intent throughout the conversation, acknowledging previous interactions to maintain context and continuity. If the user\'s message is unclear or lacks necessary details, the assistant always asks for clarification rather than making assumptions. For example, if the user\'s request is incomplete, the assistant responds with: "Could you provide more details so I can assist you better?"\n\nThe assistant asks specific follow-up questions and offers suggestions based on the user\'s 

In [5]:
# get tokenizer
from transformers import AutoTokenizer

model_name = "meta-llama/Llama-3.2-3B-Instruct"
# model_name = "meta-llama/Llama-3.1-8B-Instruct"
# model_name = "Qwen/Qwen3-1.7B"   # <-- this one is not working: assert failed

tok = AutoTokenizer.from_pretrained(model_name)

is_eval = False
tok.padding_side, tok.pad_token = ("left" if is_eval else "right"), tok.eos_token

def process(row):
    reference = tok.apply_chat_template(row["prompt"] + [{'role': 'assistant', 'content': row["chosen"]}], tokenize=False)
    row["prompt"] = tok.apply_chat_template(row["prompt"], tokenize=False, add_generation_prompt=True)
    row["chosen"] = row["chosen"].strip() + tok.eos_token
    row["rejected"] = row["rejected"].strip() +  tok.eos_token
    assert row["prompt"] + row["chosen"] == reference
    return row

# def process(row):
#     messages = row["prompt"]
#     prompt_str = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

#     chosen_body = row["chosen"].strip()
#     rejected_body = row["rejected"].strip()

#     ref_chosen = tok.apply_chat_template(messages + [{'role': 'assistant', 'content': chosen_body}], tokenize=False)
#     ref_rejected = tok.apply_chat_template(messages + [{'role': 'assistant', 'content': rejected_body}], tokenize=False)

#     row["prompt"] = prompt_str
#     row["chosen"] = ref_chosen[len(prompt_str):]
#     row["rejected"] = ref_rejected[len(prompt_str):]

#     assert row["prompt"] + row["chosen"] == ref_chosen
#     return row

dpo_ds["train"] = dpo_ds["train"].map(process, load_from_cache_file=False)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Map:   0%|          | 0/111 [00:00<?, ? examples/s]